In [ ]:
# import libraries
import numpy as np
import sys
import psi4
from helper_PFCI import PFHamiltonianGenerator
np.set_printoptions(threshold=sys.maxsize)
psi4.core.set_output_file('output.dat', False)
import time


In [ ]:
# setup basic arguments for qed-ci calculation
mol_str = """
Li
H 1 1.8
symmetry c1
"""

options_dict = {
    "basis": "sto-3g",
    "scf_type": "pk",
    "e_convergence": 1e-10,
    "d_convergence": 1e-10,
}

cavity_dict = {
    'omega_value' : 0.12086,
    'lambda_vector' : np.array([0, 0, 0.05]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}


cavity_free_dict = {
    'omega_value' : 0.0,
    'lambda_vector' : np.array([0, 0, 0.0]),
    'ci_level' : 'fci',   
    'full_diagonalization' : True,
    'number_of_photons' : 0, 
}



In [ ]:

N_el = 75
N_ph = 20
omega = 0.12086
lambda_vector = np.array([0., 0., 0.05])


E_array = np.zeros(N_el)

# run cas
LiH = PFHamiltonianGenerator(mol_str, options_dict, cavity_free_dict)

# sort singlet states
singlet_indices = LiH.sort_dipole_allowed_states(N_el)
E_array = LiH.CIeigs[singlet_indices]

# compute dipole matrix elements between singlet states
mu_array = LiH.compute_dipole_moments(singlet_indices)

# build pcqed hamiltonian with old loop-based methods
slow_start = time.time()
LiH.build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, mu_array)
slow_end = time.time()

# store the eigenvalues from old build
slow_eigs = np.copy(LiH.PCQED_eigs)

# build pcqed hamiltonian with faster method
fast_start = time.time()
LiH.fast_build_pcqed_pf_hamiltonian(N_el, N_ph, omega, lambda_vector, E_array, mu_array)
fast_end = time.time()

# store the eigenvaluse from fast build
fast_eigs = np.copy(LiH.PCQED_eigs)

# make sure all eigenvalues match from old approach to fast approach
assert np.allclose(slow_eigs, fast_eigs)

# print time associated with each approach
print(F"Slow build took {slow_end-slow_start} seconds")
print(F"Fast build took {fast_end-fast_start} seconds")




# Notes on the matrix blocks

\begin{align}\label{EQN:projected_matrix}
{\bf \mathcal{H}} =
&\begin{bmatrix}
{\bf E} + {\bf D}   & -\sqrt{\frac{\omega}{2}} {\bf d}  & 0 & \dots & 0 & 0 \\
-\sqrt{\frac{\omega}{2}}{\bf d} & {\bf E} + {\bf D} + {\bf \Omega}   & -\sqrt{\omega} {\bf d}  & \dots & 0 & 0 \\
0   &  -\sqrt{\omega}{\bf d} & {\bf E} + {\bf D} + 2{\bf \Omega} & \dots & 0  & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0      &      0 &   0    &  \dots     & {\bf E} + {\bf D} + (N-1){\bf \Omega}    & -\sqrt{\frac{N \omega}{2}} {\bf d} \\
0      &      0 &   0    &  \dots     & -\sqrt{\frac{N \omega}{2}}{\bf d}     & {\bf E} + {\bf D} + N{\bf \Omega}. 
\end{bmatrix}
\end{align}


Note that there are basically 4 different matrices with shape $(N_{el}, N_{el})$ that appear in this Hamiltonian matrix.  The following outlines the equations for the elements of each matrix along with one or more strategies to assemble them.  

### E matrix
#### Math
$$ E_{\alpha \beta} = \langle \psi_{\alpha} | \mathcal{H}_{el} | \psi_{\beta} \rangle = E_{\beta} \delta_{\alpha \beta} $$ 
where $E_{\beta}$ denote the electronic energy eigenvalues of the molecular system.
#### Code
If the energy eigenvalues are stored in an array called `E_array`, then we can build ${\bf E}$ by multiplying these values by an $(N_{el}, N_{el})$ identity matrix $\mathbb{I}$.

##### build N_el x N_el identity matrix
`_I = np.eye(N_el)`

##### build N_el x N_el _E matrix
`_E = E_array * _I`




In [ ]:
# code to build E matrix goes here

### $\Omega$ matrix
#### Math
$$ \Omega_{\alpha \beta} = \langle \psi_{\alpha} | \omega | \psi_{\beta} \rangle  = \omega \delta_{\alpha \beta}$$ 

#### Code
If the photon frequency is stored in the variable `omega`, then the $(N_{el},N_{el})$ matrix `_O` can be build 
by multiplying the $\mathbb{I}$ by $omega$:

##### build N_el x N_el _O matrix
`_O = omega * _I`


In [ ]:
# code to build Omega matrix goes here

### d matrix
#### Math
$$ d_{\alpha \beta} = \lambda \cdot  \langle \psi_{\alpha} | \mathcal{\mu} | \psi_{\beta} \rangle = \lambda_x \mathcal{\mu}_{x, \alpha \beta} +  \lambda_y \mathcal{\mu}_{y, \alpha \beta} +  \lambda_z \mathcal{\mu}_{z, \alpha \beta} $$ 
where $\mathcal{\mu}_{x, \alpha \beta}$ denotes the x-component of the (transition) dipole moment between molecular electronic state $\psi_{\alpha}$ and $\psi_{\beta}$.
#### Code
If the dipole matrix elements are stored in a $(N_{el}, N_{el}, 3)$  array called `mu_array` and the $\lambda$ vector is stored in an 3-element array called `\lambda_vector`, then we can build the
$(N_{el}, N_{el})$ array ${\bf d}$ by [contraction](https://en.wikipedia.org/wiki/Tensor_contraction) using `np.einsum()`:

##### build N_el x N_el _d matrix
`_d = np.einsum("k,ijk->ij", lambda_vector, mu_array)`

In [ ]:
# code to build the d matrix goes here

### D matrix
#### Math
$$ D_{\alpha \beta} = \frac{1}{2} \sum_{\gamma} d_{\alpha \gamma} d_{\gamma \beta} $$ 

#### Code
We have the $(N_{el}, N_{el})$ elements of the ${\bf d}$ array stored in `_d`.  We can then build `_D`
using matrix-matrix multiplication as follows:

`_D = _d @ _d`

or using einsum as follows:

`_D = np.einsum("ik,kj->ij",_d, _d)`
